In [60]:
import pandas as pd
import numpy as np

# import files
county_df = pd.read_csv('us-counties.csv', parse_dates=['date'])
pop_df = pd.read_csv('or_census.csv')

print(county_df.shape, pop_df.shape)
print(county_df.columns)
print(pop_df.columns)

(1066487, 6) (36, 13)
Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')
Index(['County', 'Census 2010', 'Estimates 2010', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')


In [61]:
# only keep Oregon data
county_df = county_df[county_df['state'] == 'Oregon']
county_df = county_df[county_df['county'] != 'Unknown']
print(county_df.shape)

# find missing dates
county_df = county_df.set_index(['county'])
print(county_df.index.unique())
start = county_df['date'].min()
end = county_df['date'].max()

for county in county_df.index.unique():
    c_df = county_df.loc[county]
    missing = pd.date_range(start = start, end = end).difference(c_df['date'])
    
    if len(missing) > 0:
        # add missing values as NaN
        for item in missing:
            # create row
            new_row = pd.Series([item, county_df.loc[county,'state'][0], county_df.loc[county,'fips'][0], np.NaN, np.NaN], index=['date','state','fips','cases','deaths'], name=county)
            #new_row.index.name = county
            county_df = county_df.append(new_row)
    
county_df.reset_index(inplace=True)
print(county_df.shape)

(11865, 6)
Index(['Washington', 'Jackson', 'Klamath', 'Douglas', 'Marion', 'Multnomah',
       'Deschutes', 'Linn', 'Polk', 'Umatilla', 'Clackamas', 'Yamhill',
       'Benton', 'Lane', 'Grant', 'Union', 'Josephine', 'Hood River',
       'Clatsop', 'Lincoln', 'Tillamook', 'Wasco', 'Columbia', 'Morrow',
       'Malheur', 'Wallowa', 'Crook', 'Sherman', 'Curry', 'Coos', 'Jefferson',
       'Harney', 'Baker', 'Lake', 'Gilliam', 'Wheeler'],
      dtype='object', name='county')
(13104, 6)


In [62]:
# calculate new deaths and new cases per day
county_df['new_cases'] = np.NaN
county_df['new_deaths'] = np.NaN

# group by state, county, date
county_df = county_df.set_index(['county','date'])
county_df = county_df.sort_index(ascending=True)

for ccounty, cdate in county_df.index:
    if cdate == pd.to_datetime('20200228',format='%Y%m%d'):
        county_df.loc[(ccounty, cdate),'new_cases'] = county_df.loc[(ccounty, cdate),'cases']
        county_df.loc[(ccounty, cdate),'new_deaths'] = county_df.loc[(ccounty, cdate),'deaths']
    else:
        # get previous date
        pdate = cdate - pd.Timedelta(days=1)
        county_df.loc[(ccounty, cdate),'new_cases'] = county_df.loc[(ccounty, cdate),'cases'] - county_df.loc[(ccounty, pdate),'cases']
        county_df.loc[(ccounty, cdate),'new_deaths'] = county_df.loc[(ccounty, cdate),'deaths'] - county_df.loc[(ccounty, pdate),'deaths']

print(county_df)

2020-02-28 00:00:00
state          Oregon
fips          41001.0
cases             NaN
deaths            NaN
new_cases         NaN
new_deaths        NaN
Name: (Baker, 2020-05-05 00:00:00), dtype: object
Baker 2020-02-28 00:00:00
Baker 2020-02-29 00:00:00
Baker 2020-03-01 00:00:00
Baker 2020-03-02 00:00:00
Baker 2020-03-03 00:00:00
Baker 2020-03-04 00:00:00
Baker 2020-03-05 00:00:00
Baker 2020-03-06 00:00:00
Baker 2020-03-07 00:00:00
Baker 2020-03-08 00:00:00
Baker 2020-03-09 00:00:00
Baker 2020-03-10 00:00:00
Baker 2020-03-11 00:00:00
Baker 2020-03-12 00:00:00
Baker 2020-03-13 00:00:00
Baker 2020-03-14 00:00:00
Baker 2020-03-15 00:00:00
Baker 2020-03-16 00:00:00
Baker 2020-03-17 00:00:00
Baker 2020-03-18 00:00:00
Baker 2020-03-19 00:00:00
Baker 2020-03-20 00:00:00
Baker 2020-03-21 00:00:00
Baker 2020-03-22 00:00:00
Baker 2020-03-23 00:00:00
Baker 2020-03-24 00:00:00
Baker 2020-03-25 00:00:00
Baker 2020-03-26 00:00:00
Baker 2020-03-27 00:00:00
Baker 2020-03-28 00:00:00
Baker 2020-03-29 0

Baker 2021-02-15 00:00:00
Baker 2021-02-16 00:00:00
Baker 2021-02-17 00:00:00
Baker 2021-02-18 00:00:00
Baker 2021-02-19 00:00:00
Baker 2021-02-20 00:00:00
Baker 2021-02-21 00:00:00
Baker 2021-02-22 00:00:00
Baker 2021-02-23 00:00:00
Baker 2021-02-24 00:00:00
Baker 2021-02-25 00:00:00
Benton 2020-02-28 00:00:00
Benton 2020-02-29 00:00:00
Benton 2020-03-01 00:00:00
Benton 2020-03-02 00:00:00
Benton 2020-03-03 00:00:00
Benton 2020-03-04 00:00:00
Benton 2020-03-05 00:00:00
Benton 2020-03-06 00:00:00
Benton 2020-03-07 00:00:00
Benton 2020-03-08 00:00:00
Benton 2020-03-09 00:00:00
Benton 2020-03-10 00:00:00
Benton 2020-03-11 00:00:00
Benton 2020-03-12 00:00:00
Benton 2020-03-13 00:00:00
Benton 2020-03-14 00:00:00
Benton 2020-03-15 00:00:00
Benton 2020-03-16 00:00:00
Benton 2020-03-17 00:00:00
Benton 2020-03-18 00:00:00
Benton 2020-03-19 00:00:00
Benton 2020-03-20 00:00:00
Benton 2020-03-21 00:00:00
Benton 2020-03-22 00:00:00
Benton 2020-03-23 00:00:00
Benton 2020-03-24 00:00:00
Benton 2020-

Benton 2021-02-04 00:00:00
Benton 2021-02-05 00:00:00
Benton 2021-02-06 00:00:00
Benton 2021-02-07 00:00:00
Benton 2021-02-08 00:00:00
Benton 2021-02-09 00:00:00
Benton 2021-02-10 00:00:00
Benton 2021-02-11 00:00:00
Benton 2021-02-12 00:00:00
Benton 2021-02-13 00:00:00
Benton 2021-02-14 00:00:00
Benton 2021-02-15 00:00:00
Benton 2021-02-16 00:00:00
Benton 2021-02-17 00:00:00
Benton 2021-02-18 00:00:00
Benton 2021-02-19 00:00:00
Benton 2021-02-20 00:00:00
Benton 2021-02-21 00:00:00
Benton 2021-02-22 00:00:00
Benton 2021-02-23 00:00:00
Benton 2021-02-24 00:00:00
Benton 2021-02-25 00:00:00
Clackamas 2020-02-28 00:00:00
Clackamas 2020-02-29 00:00:00
Clackamas 2020-03-01 00:00:00
Clackamas 2020-03-02 00:00:00
Clackamas 2020-03-03 00:00:00
Clackamas 2020-03-04 00:00:00
Clackamas 2020-03-05 00:00:00
Clackamas 2020-03-06 00:00:00
Clackamas 2020-03-07 00:00:00
Clackamas 2020-03-08 00:00:00
Clackamas 2020-03-09 00:00:00
Clackamas 2020-03-10 00:00:00
Clackamas 2020-03-11 00:00:00
Clackamas 2020-0

Clackamas 2020-11-23 00:00:00
Clackamas 2020-11-24 00:00:00
Clackamas 2020-11-25 00:00:00
Clackamas 2020-11-26 00:00:00
Clackamas 2020-11-27 00:00:00
Clackamas 2020-11-28 00:00:00
Clackamas 2020-11-29 00:00:00
Clackamas 2020-11-30 00:00:00
Clackamas 2020-12-01 00:00:00
Clackamas 2020-12-02 00:00:00
Clackamas 2020-12-03 00:00:00
Clackamas 2020-12-04 00:00:00
Clackamas 2020-12-05 00:00:00
Clackamas 2020-12-06 00:00:00
Clackamas 2020-12-07 00:00:00
Clackamas 2020-12-08 00:00:00
Clackamas 2020-12-09 00:00:00
Clackamas 2020-12-10 00:00:00
Clackamas 2020-12-11 00:00:00
Clackamas 2020-12-12 00:00:00
Clackamas 2020-12-13 00:00:00
Clackamas 2020-12-14 00:00:00
Clackamas 2020-12-15 00:00:00
Clackamas 2020-12-16 00:00:00
Clackamas 2020-12-17 00:00:00
Clackamas 2020-12-18 00:00:00
Clackamas 2020-12-19 00:00:00
Clackamas 2020-12-20 00:00:00
Clackamas 2020-12-21 00:00:00
Clackamas 2020-12-22 00:00:00
Clackamas 2020-12-23 00:00:00
Clackamas 2020-12-24 00:00:00
Clackamas 2020-12-25 00:00:00
Clackamas 

Clatsop 2020-11-02 00:00:00
Clatsop 2020-11-03 00:00:00
Clatsop 2020-11-04 00:00:00
Clatsop 2020-11-05 00:00:00
Clatsop 2020-11-06 00:00:00
Clatsop 2020-11-07 00:00:00
Clatsop 2020-11-08 00:00:00
Clatsop 2020-11-09 00:00:00
Clatsop 2020-11-10 00:00:00
Clatsop 2020-11-11 00:00:00
Clatsop 2020-11-12 00:00:00
Clatsop 2020-11-13 00:00:00
Clatsop 2020-11-14 00:00:00
Clatsop 2020-11-15 00:00:00
Clatsop 2020-11-16 00:00:00
Clatsop 2020-11-17 00:00:00
Clatsop 2020-11-18 00:00:00
Clatsop 2020-11-19 00:00:00
Clatsop 2020-11-20 00:00:00
Clatsop 2020-11-21 00:00:00
Clatsop 2020-11-22 00:00:00
Clatsop 2020-11-23 00:00:00
Clatsop 2020-11-24 00:00:00
Clatsop 2020-11-25 00:00:00
Clatsop 2020-11-26 00:00:00
Clatsop 2020-11-27 00:00:00
Clatsop 2020-11-28 00:00:00
Clatsop 2020-11-29 00:00:00
Clatsop 2020-11-30 00:00:00
Clatsop 2020-12-01 00:00:00
Clatsop 2020-12-02 00:00:00
Clatsop 2020-12-03 00:00:00
Clatsop 2020-12-04 00:00:00
Clatsop 2020-12-05 00:00:00
Clatsop 2020-12-06 00:00:00
Clatsop 2020-12-07 0

...

Wasco 2020-09-04 00:00:00
Wasco 2020-09-05 00:00:00
Wasco 2020-09-06 00:00:00
Wasco 2020-09-07 00:00:00
Wasco 2020-09-08 00:00:00
Wasco 2020-09-09 00:00:00
Wasco 2020-09-10 00:00:00
Wasco 2020-09-11 00:00:00
Wasco 2020-09-12 00:00:00
Wasco 2020-09-13 00:00:00
Wasco 2020-09-14 00:00:00
Wasco 2020-09-15 00:00:00
Wasco 2020-09-16 00:00:00
Wasco 2020-09-17 00:00:00
Wasco 2020-09-18 00:00:00
Wasco 2020-09-19 00:00:00
Wasco 2020-09-20 00:00:00
Wasco 2020-09-21 00:00:00
Wasco 2020-09-22 00:00:00
Wasco 2020-09-23 00:00:00
Wasco 2020-09-24 00:00:00
Wasco 2020-09-25 00:00:00
Wasco 2020-09-26 00:00:00
Wasco 2020-09-27 00:00:00
Wasco 2020-09-28 00:00:00
Wasco 2020-09-29 00:00:00
Wasco 2020-09-30 00:00:00
Wasco 2020-10-01 00:00:00
Wasco 2020-10-02 00:00:00
Wasco 2020-10-03 00:00:00
Wasco 2020-10-04 00:00:00
Wasco 2020-10-05 00:00:00
Wasco 2020-10-06 00:00:00
Wasco 2020-10-07 00:00:00
Wasco 2020-10-08 00:00:00
Wasco 2020-10-09 00:00:00
Wasco 2020-10-10 00:00:00
Wasco 2020-10-11 00:00:00
Wasco 2020-1

Washington 2020-07-29 00:00:00
Washington 2020-07-30 00:00:00
Washington 2020-07-31 00:00:00
Washington 2020-08-01 00:00:00
Washington 2020-08-02 00:00:00
Washington 2020-08-03 00:00:00
Washington 2020-08-04 00:00:00
Washington 2020-08-05 00:00:00
Washington 2020-08-06 00:00:00
Washington 2020-08-07 00:00:00
Washington 2020-08-08 00:00:00
Washington 2020-08-09 00:00:00
Washington 2020-08-10 00:00:00
Washington 2020-08-11 00:00:00
Washington 2020-08-12 00:00:00
Washington 2020-08-13 00:00:00
Washington 2020-08-14 00:00:00
Washington 2020-08-15 00:00:00
Washington 2020-08-16 00:00:00
Washington 2020-08-17 00:00:00
Washington 2020-08-18 00:00:00
Washington 2020-08-19 00:00:00
Washington 2020-08-20 00:00:00
Washington 2020-08-21 00:00:00
Washington 2020-08-22 00:00:00
Washington 2020-08-23 00:00:00
Washington 2020-08-24 00:00:00
Washington 2020-08-25 00:00:00
Washington 2020-08-26 00:00:00
Washington 2020-08-27 00:00:00
Washington 2020-08-28 00:00:00
Washington 2020-08-29 00:00:00
Washingt

Wheeler 2020-05-23 00:00:00
Wheeler 2020-05-24 00:00:00
Wheeler 2020-05-25 00:00:00
Wheeler 2020-05-26 00:00:00
Wheeler 2020-05-27 00:00:00
Wheeler 2020-05-28 00:00:00
Wheeler 2020-05-29 00:00:00
Wheeler 2020-05-30 00:00:00
Wheeler 2020-05-31 00:00:00
Wheeler 2020-06-01 00:00:00
Wheeler 2020-06-02 00:00:00
Wheeler 2020-06-03 00:00:00
Wheeler 2020-06-04 00:00:00
Wheeler 2020-06-05 00:00:00
Wheeler 2020-06-06 00:00:00
Wheeler 2020-06-07 00:00:00
Wheeler 2020-06-08 00:00:00
Wheeler 2020-06-09 00:00:00
Wheeler 2020-06-10 00:00:00
Wheeler 2020-06-11 00:00:00
Wheeler 2020-06-12 00:00:00
Wheeler 2020-06-13 00:00:00
Wheeler 2020-06-14 00:00:00
Wheeler 2020-06-15 00:00:00
Wheeler 2020-06-16 00:00:00
Wheeler 2020-06-17 00:00:00
Wheeler 2020-06-18 00:00:00
Wheeler 2020-06-19 00:00:00
Wheeler 2020-06-20 00:00:00
Wheeler 2020-06-21 00:00:00
Wheeler 2020-06-22 00:00:00
Wheeler 2020-06-23 00:00:00
Wheeler 2020-06-24 00:00:00
Wheeler 2020-06-25 00:00:00
Wheeler 2020-06-26 00:00:00
Wheeler 2020-06-27 0

Yamhill 2020-04-03 00:00:00
Yamhill 2020-04-04 00:00:00
Yamhill 2020-04-05 00:00:00
Yamhill 2020-04-06 00:00:00
Yamhill 2020-04-07 00:00:00
Yamhill 2020-04-08 00:00:00
Yamhill 2020-04-09 00:00:00
Yamhill 2020-04-10 00:00:00
Yamhill 2020-04-11 00:00:00
Yamhill 2020-04-12 00:00:00
Yamhill 2020-04-13 00:00:00
Yamhill 2020-04-14 00:00:00
Yamhill 2020-04-15 00:00:00
Yamhill 2020-04-16 00:00:00
Yamhill 2020-04-17 00:00:00
Yamhill 2020-04-18 00:00:00
Yamhill 2020-04-19 00:00:00
Yamhill 2020-04-20 00:00:00
Yamhill 2020-04-21 00:00:00
Yamhill 2020-04-22 00:00:00
Yamhill 2020-04-23 00:00:00
Yamhill 2020-04-24 00:00:00
Yamhill 2020-04-25 00:00:00
Yamhill 2020-04-26 00:00:00
Yamhill 2020-04-27 00:00:00
Yamhill 2020-04-28 00:00:00
Yamhill 2020-04-29 00:00:00
Yamhill 2020-04-30 00:00:00
Yamhill 2020-05-01 00:00:00
Yamhill 2020-05-02 00:00:00
Yamhill 2020-05-03 00:00:00
Yamhill 2020-05-04 00:00:00
Yamhill 2020-05-05 00:00:00
Yamhill 2020-05-06 00:00:00
Yamhill 2020-05-07 00:00:00
Yamhill 2020-05-08 0

Yamhill 2021-01-31 00:00:00
Yamhill 2021-02-01 00:00:00
Yamhill 2021-02-02 00:00:00
Yamhill 2021-02-03 00:00:00
Yamhill 2021-02-04 00:00:00
Yamhill 2021-02-05 00:00:00
Yamhill 2021-02-06 00:00:00
Yamhill 2021-02-07 00:00:00
Yamhill 2021-02-08 00:00:00
Yamhill 2021-02-09 00:00:00
Yamhill 2021-02-10 00:00:00
Yamhill 2021-02-11 00:00:00
Yamhill 2021-02-12 00:00:00
Yamhill 2021-02-13 00:00:00
Yamhill 2021-02-14 00:00:00
Yamhill 2021-02-15 00:00:00
Yamhill 2021-02-16 00:00:00
Yamhill 2021-02-17 00:00:00
Yamhill 2021-02-18 00:00:00
Yamhill 2021-02-19 00:00:00
Yamhill 2021-02-20 00:00:00
Yamhill 2021-02-21 00:00:00
Yamhill 2021-02-22 00:00:00
Yamhill 2021-02-23 00:00:00
Yamhill 2021-02-24 00:00:00
Yamhill 2021-02-25 00:00:00
                     state     fips   cases  deaths  new_cases  new_deaths
county  date                                                              
Baker   2020-02-28  Oregon  41001.0     NaN     NaN        NaN         NaN
        2020-02-29  Oregon  41001.0     NaN    

In [63]:
# export to csv
county_df.to_csv('county_cleaned.csv', index=False)